<a href="https://colab.research.google.com/github/elvissoares/EQE595-SimMol/blob/main/notebooks/6_Soluto_e_Solvente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para usar o OpenMM no Google Colab devemos fazer o seguinte passo:
1. Instalar o `openmm[cuda12]` e o `mdtraj` via `pip`

In [ ]:
!pip install openmm[cuda12] mdtraj

2. Testando se a instalação deu certo e quais `Platform` estão disponíveis

In [ ]:
!python -m openmm.testInstallation

# Aula Prática 06 - Dinâmica Molecular de um Soluto em Água

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares)

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

Qual molécula iremos simular?

In [ ]:
!wget https://files.rcsb.org/download/1AKI.pdb

In [ ]:
pdb = PDBFile('1AKI.pdb')

Condição termodinâmica

In [ ]:
Temperatura = 298.15 * kelvin # Temperatura em Kelvin
Pressao = 1 * bar # Pressão em bar

Escolhendo os arquivos de campo de força `Amber19`

In [ ]:
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

Cria a topolgia com moléculas de água ao redor (solvente)

In [ ]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()
residues=modeller.addHydrogens(forcefield)

modeller.addSolvent(forcefield, padding = 1.0*nanometer)

PDBFile.writeFile(modeller.topology, modeller.positions, open(f'initial_solute.pdb', 'w'))

Definindo o `system`, o `platform`, o `integrator` e o `simulation`

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)

# Plataforma (OpenCL se GPU disponível)
platform = Platform.getPlatformByName('OpenCL')
# Integrador
integrator = LangevinMiddleIntegrator(Temperatura, 1/picosecond, 4*femtoseconds)

simulation = Simulation(modeller.topology, system, integrator,platform)
simulation.context.setPositions(modeller.positions)

Minimizando a energia

In [ ]:
simulation.minimizeEnergy()

energy = simulation.context.getState(getEnergy=True).getPotentialEnergy()
print(f'Energia: {energy}')


Definindo os `reporters`

In [ ]:
# Remove reporters existentes, no caso dessa célula ser executada mais de uma vez.
simulation.reporters = []

# Escreve cada frame em arquivo PDB de trajetórias a cada 1000 passos
simulation.reporters.append(PDBReporter(f"output_solute.pdb", 1000))

# Escreve num arquivo csv a cada 100 passos
simulation.reporters.append(
    StateDataReporter(
        f"md_log_solute.csv",
        100,
        step=True,
        time=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True,
        volume=True
    )
)

# Escreve na tela a cada 1000 passos
simulation.reporters.append(
    StateDataReporter(stdout, 1000, step=True, totalEnergy=True,temperature=True,
        volume=True)
)

## Equilibração NVT

In [ ]:
simulation.context.setVelocitiesToTemperature(Temperatura)

simulation.step(10_000)

## Produção NPT

In [ ]:
system.addForce(MonteCarloBarostat(Pressao, Temperatura))
simulation.context.reinitialize(preserveState=True)

print("Running NPT")
simulation.step(250_000)

# A próxima linha é necessária para Windows para fechar os arquivos antes de serem abertos
del simulation

## Analisando a estatística dos dados

In [ ]:
df = pd.read_csv(f"md_log_solute.csv")

In [ ]:
df.head()

In [ ]:
df['Total Energy (kJ/mole)']

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True)

axs[0].plot(df['Time (ps)'],df['Potential Energy (kJ/mole)'],color='C0',label='U')
axs[0].legend(loc='best')
axs[0].set_ylabel('U (kJ/mole)')

axs[1].plot(df['Time (ps)'],df['Temperature (K)'],color='C3',label='T')
axs[1].legend(loc='best')
axs[1].set_ylabel('T (K)')

axs[2].plot(df['Time (ps)'],df['Box Volume (nm^3)'],color='C1',label='V')
axs[2].legend(loc='best')
axs[2].set_xlabel('Time (ps)')
axs[2].set_ylabel('Volume (nm$^3$)')

In [ ]:
timecut = 80
subsetT = df['Temperature (K)'][df['Time (ps)'] > timecut]
subsetU = df['Potential Energy (kJ/mole)'][df['Time (ps)'] > timecut]
subsetV = df['Box Volume (nm^3)'][df['Time (ps)'] > timecut]

In [ ]:
from IPython.display import display, Math

Umean = subsetU.mean()
sigmaU = subsetU.std()

display(Math(rf"U = {Umean:.2f} \pm {sigmaU:.2f}\ kJ/mole"))

In [ ]:
100*sigmaU/abs(Umean)

In [ ]:
Tmean = subsetT.mean() # média
sigmaT = subsetT.std() # desvio padrão

display(Math(rf"$T = {Tmean:.2f} \pm {sigmaT:.2f}\ K"))

In [ ]:
100*sigmaT/abs(Tmean)

In [ ]:
Vmean = subsetV.mean()
sigmaV = subsetV.std()

display(Math(rf"$V = {Vmean:.2f} \pm {sigmaV:.2f}\ nm^3"))

In [ ]:
100*sigmaV/abs(Vmean)

Analisando as distribuições estatísticas

In [ ]:
fig, axs = plt.subplots(1, 3,figsize=(7,3))

axs[0].hist(subsetU,bins=30,density=True)
axs[0].set_xlabel(r'$U$ (kJ/mole)')

uarray = Umean+np.arange(-5*sigmaU,5*sigmaU,0.01*sigmaU)
axs[0].plot(uarray,np.sqrt(1/(2*np.pi*sigmaU**2))*np.exp(-0.5*(uarray-Umean)**2/sigmaU**2),color='k')

axs[1].hist(subsetT,bins=30,density=True,color='C3')
axs[1].set_xlabel(r'$T$ (K)')

Tarray = Tmean + np.arange(-5*sigmaT,5*sigmaT,0.01*sigmaT)
axs[1].plot(Tarray,np.sqrt(1/(2*np.pi*sigmaT**2))*np.exp(-0.5*(Tarray-Tmean)**2/sigmaT**2),color='k')

axs[2].hist(subsetV,bins=30,density=True,color='C2')
axs[2].set_xlabel(r'$V$ (nm$^3$)')

Varray = Vmean + np.arange(-5*sigmaV,5*sigmaV,0.01*sigmaV)
axs[2].plot(Varray,np.sqrt(1/(2*np.pi*sigmaV**2))*np.exp(-0.5*(Varray-Vmean)**2/sigmaV**2),color='k')

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Faça uma estimativa do custo computacional (tempo de simulação) no _hardware_ atual para simulações longas da ordem de 1ns.
>


Tempo Simulação | Tempo Computacional |
--- | --- |
0.4 ns | 1m 50 seg
1.0 ns | 4m 40 seg
10.0 ns | ~40 min

In [ ]:
x = 110/0.4

x/60

In [ ]:
1*nanosecond/(4*femtosecond)